<a href="https://colab.research.google.com/github/Fatih120/GuildedChatExporter/blob/main/guildedchatexporter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GuildedChatExporter

Export your stuff before it's gone.

## This ONLY works on normal text channels at the moment, maybe streaming/voice channels, haven't tested. Heavy WIP.

1.   Set up a bot within the server you want to export from and create an Authentication Key for it. Paste it in the first box below.
2.   Grab the Channel ID of the channel you want to archive. Have Settings > Advanced > Developer mode enabled, then right click a channel and copy the ID: or, find the ID in the channel URL after /channel/*. Paste it below.
3.   Run the cells one by one, skipping the optional ones since you likely don't need them. Do this by clicking the play circle on the top left. Keep your Google Drive open to see your files.

Need help?
https://github.com/Fatih120/GuildedChatExporter/tree/main


[Mountain of Fatih Guilded](https://guilded.gg/MoF)

[Discord :(](https://discord.com/invite/Cy27FNfQtc)



In [ ]:
# @title Basic setup
API_KEY = "gapi_KEY==" # @param {type:"string"}
#SERVER_ID = "dlOB6yVR" # @param {type:"string"}
CHANNEL_ID = "XXXXXXXX-YYYY-ZZZZ-WWWW-YYYYYYYYYYYY " # @param {type:"string"}
SAVE_DIRECTORY = "/content/drive/MyDrive/guildedchatexporter/" # @param {type:"string"}
import requests
import json
import os
import re

headers = {
    'Authorization': f'Bearer {API_KEY}',
    'Accept': 'application/json',
    'Content-Type': 'application/json'
}
SERVER_URL = f'https://www.guilded.gg/api/v1/servers/{SERVER_ID}'
CHANNEL_URL = f'https://www.guilded.gg/api/v1/channels/{CHANNEL_ID}/messages'


In [ ]:
# @title Set-Up Google Drive (It's probably already done for you)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title Get Messages
def get_channel_messages(channel_id, after=None, limit=100):
    params = {
        'limit': limit,
        'after': after
    }
    url = f'https://www.guilded.gg/api/v1/channels/{channel_id}/messages'
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    return response.json()

def fetch_all_messages(channel_id):
    all_messages = []
    after = "2015-01-01T00:00:00.000Z"

    while True:
        data = get_channel_messages(channel_id, after=after, limit=100)
        messages = data.get('messages', [])
        all_messages.extend(messages)

        if len(messages) < 100:
            break

        after = messages[-1]['createdAt']

    return all_messages

all_messages = fetch_all_messages(CHANNEL_ID)

print(f"Found {len(all_messages)} messages.")


In [ ]:
# @title (Optional) Print all messages and save raw results
Print_JSON = False # @param {type:"boolean"}
Save_JSON = True # @param {type:"boolean"}
if Print_JSON:
    print(json.dumps(all_messages, indent=4))
if Save_JSON:
    with open(f'{SAVE_DIRECTORY}{CHANNEL_ID}.json', 'w', encoding='utf-8') as file:
        json.dump(all_messages, file, indent=4)
        print(f"JSON of all messages saved to {SAVE_DIRECTORY}{CHANNEL_ID}.json")

In [ ]:
# @title (Optional) Save mildly-formatted .txt chat log

with open(f'{SAVE_DIRECTORY}{CHANNEL_ID}.txt', 'w', encoding='utf-8') as file:
    for message in all_messages:
        created_by = message['createdBy']
        created_at = message['createdAt']
        content = message['content']
        formatted_message = f"[{created_by}] ({created_at}): {content}\n"
        file.write(formatted_message)

print("Chat log saved")

In [ ]:
# @title HTML Chat Exporter
# @markdown Has the base HTML and CSS information which you can edit. Just click RUN to set the info for the next step.
# @markdown
# @markdown This will have a flag in the future for if you don't want to download/show attachments, but why?
## HTML Template

CSS = """
@font-face {
  font-family: "Builder Sans";
  src: url("https://www.guilded.gg/fonts/BuilderSans-Regular.woff2") format('woff2');
}

#chat-log {
  background-color: #373943;
}

.message {
  color: #ececee;
  font-size: 15px;
  line-height: 145%;
  font-weight: normal;
  font-family: 'Builder Sans', Tahoma, sans-serif;

  padding-bottom: 3px;

  margin-top: 12px;
  padding-top: 4px;
  min-height: 44px;

  padding: 0 16px;
  flex-shrink: 0;
  transition: background-color 140ms ease;
  position: relative;
  contain: layout;
}

.message:hover {
  background-color: #31333c;
}

.created-at {
  margin-right: 8px;
  height: 11px;
  color: #a3a3ac;
  font-size: 13px;
  line-height: 120%;
  font-weight: normal;
  white-space: nowrap;
  pointer-events: auto;
}

.created-by {
  font-size: 15px;
  line-height: 145%;
  font-weight: 700;
  white-space: nowrap;
  pointer-events: auto;
}
"""

HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>Chat Log</title>
    <style>
        {CSS}
    </style>
</head>
<body>
    <h1>Chat Log</h1>
    <div id="chat-log">
        {messages}
    </div>
</body>
</html>
"""

## Message Template
MESSAGE_TEMPLATE = """
<div class="message">
    <span class="created-by">{created_by}</span>
    <span class="created-at">({created_at})</span>
    <p class="content">{content}</p>
</div>
"""

print("HTML ready to print")


In [ ]:
# @title Parse and save results into HTML file!

from urllib.parse import urlparse, unquote

formatted_messages = []
for message in all_messages:
    created_by = message['createdBy']
    created_at = message['createdAt']
    content = message['content']

    # Find inline attachments and replace them with relative links or embedded content
    inline_attachments = re.findall(r'!\[\]\((.*?)\)', content)
    for attachment in inline_attachments:
        parsed_url = urlparse(attachment)
        filename = os.path.basename(unquote(parsed_url.path))
        relative_path = f'{CHANNEL_ID}/attachments/{filename}'

        if filename.endswith('.webp'):
            content = content.replace(f'![]({attachment})', f'<img src="{relative_path}" alt="{filename}">')
        elif filename.endswith('.mp4'):
            content = content.replace(f'![]({attachment})', f'<video controls><source src="{relative_path}" type="video/webm">Your browser does not support the video tag.</video>')
        else:
            content = content.replace(f'![]({attachment})', f'<a href="{relative_path}" target="_blank">{filename}</a>')

    formatted_message = MESSAGE_TEMPLATE.format(
        created_by=created_by,
        created_at=created_at,
        content=content
    )
    formatted_messages.append(formatted_message)

html_content = HTML_TEMPLATE.format(CSS=CSS, messages='\n'.join(formatted_messages))

with open(os.path.join(SAVE_DIRECTORY, f'{CHANNEL_ID}.html'), 'w', encoding='utf-8') as file:
    file.write(html_content)

print(f"HTML file saved! Get it at {SAVE_DIRECTORY}{CHANNEL_ID}.html! Run the next step to get your attachments.")

In [ ]:
# @title Download all Attachments and Uploads from channel


# Create the attachments folder if it doesn't exist
attachments_dir = os.path.join(SAVE_DIRECTORY, CHANNEL_ID, 'attachments', )
os.makedirs(attachments_dir, exist_ok=True)

def download_attachments(messages, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    url_pattern = r'!\[(.*?)\]\((.*?)\)'

    for message in messages:
        content = message['content']
        matches = re.findall(url_pattern, content)

        for match in matches:
            url = match[1]
            parsed_url = urlparse(url)
            clean_filename = os.path.basename(unquote(parsed_url.path))
            filename = os.path.join(output_dir, clean_filename)

            try:
                response = requests.get(url)
                response.raise_for_status()

                with open(filename, 'wb') as file:
                    file.write(response.content)

                print(f"Downloaded attachment from {url} to {filename}")
            except requests.exceptions.RequestException as e:
                print(f"Error downloading attachment from {url}: {e}")

all_messages = fetch_all_messages(CHANNEL_ID)
output_dir = f'{SAVE_DIRECTORY}{CHANNEL_ID}'+'/attachments'
download_attachments(all_messages, output_dir)